# Training Bank Model - Qwen3-VL-2B-Instruct

Notebook này dùng để fine-tune model Qwen3-VL-2B-Instruct cho tác vụ ngân hàng.


## 1. Cài đặt Dependencies


In [1]:
# Cài đặt các thư viện cần thiết
%pip install -q git+https://github.com/huggingface/transformers
%pip install -q accelerate peft bitsandbytes datasets pillow torch torchvision
%pip install -q trl tensorboard


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 5.1.2 requires transformers<5.0.0,>=4.41.0, but you have transformers 5.0.0.dev0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.5/465.5 kB 36.1 MB/s eta 0:00:00


## 2. Import Libraries


In [2]:
import torch
from transformers import (
    Qwen3VLForConditionalGeneration,
    AutoProcessor,
    TrainingArguments,
    Trainer,
    BitsAndBytesConfig
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from datasets import Dataset, load_dataset
from PIL import Image
import os
from huggingface_hub import login, HfApi
import json


## 3. Authentication với Hugging Face (nếu cần upload model)


In [3]:
# Uncomment và thêm token của bạn nếu muốn upload model
# login(token="YOUR_HF_TOKEN")


## 4. Load Model và Processor


In [4]:
# Cấu hình quantization để tiết kiệm bộ nhớ
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

# Load model từ Hugging Face
model_name = "hainguyen306201/bank-model-2b"

model = Qwen3VLForConditionalGeneration.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True
)

# Load processor
processor = AutoProcessor.from_pretrained(model_name, trust_remote_code=True)

print("Model và processor đã được load thành công!")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


Loading weights:   0%|          | 0/625 [00:00<?, ?it/s]

Model và processor đã được load thành công!


## 5. Setup LoRA cho Efficient Training


In [5]:
# Chuẩn bị model cho training
model = prepare_model_for_kbit_training(model)

# Cấu hình LoRA
lora_config = LoraConfig(
    r=16,  # Rank
    lora_alpha=32,  # LoRA alpha
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],  # Có thể cần điều chỉnh
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

# Áp dụng LoRA
model = get_peft_model(model, lora_config)

# In số lượng parameters có thể train
model.print_trainable_parameters()


trainable params: 6,422,528 || all params: 2,133,954,560 || trainable%: 0.3010


## 6. Chuẩn bị Dataset


In [6]:
# Ví dụ dataset format cho vision-language model
# Bạn có thể load dataset từ Hugging Face hoặc tạo từ local files

# Option 1: Tạo dataset từ local (ví dụ)
def create_sample_dataset():
    """
    Tạo dataset mẫu. Format:
    - image: đường dẫn hoặc URL của ảnh
    - text: câu hỏi hoặc instruction
    - answer: câu trả lời mong muốn
    """
    data = [
        {
            "image": "https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen-VL/assets/demo.jpeg",
            "text": "Mô tả ảnh này.",
            "answer": "Đây là một bức ảnh mẫu."
        },
        # Thêm nhiều samples hơn ở đây
    ]
    return Dataset.from_list(data)

# Option 2: Load từ Hugging Face dataset
# dataset = load_dataset("your_dataset_name", split="train")

# Sử dụng dataset mẫu (thay thế bằng dataset thực của bạn)
train_dataset = create_sample_dataset()

print(f"Dataset size: {len(train_dataset)}")


Dataset size: 1


In [7]:
# Hàm preprocessing cho dataset
def preprocess_function(examples):
    """
    Preprocess data cho model
    """
    messages = []
    
    for i in range(len(examples["text"])):
        # Tạo message format cho Qwen3-VL
        message = [
            {
                "role": "user",
                "content": [
                    {"type": "image", "image": examples["image"][i]},
                    {"type": "text", "text": examples["text"][i]},
                ],
            },
            {
                "role": "assistant",
                "content": [{"type": "text", "text": examples["answer"][i]}],
            },
        ]
        messages.append(message)
    
    # Apply chat template
    inputs = processor.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=False,
        return_dict=True,
        return_tensors="pt",
    )
    
    # Tạo labels (copy input_ids, mask padding tokens)
    labels = inputs["input_ids"].clone()
    
    return {
        "input_ids": inputs["input_ids"].squeeze(),
        "attention_mask": inputs["attention_mask"].squeeze(),
        "labels": labels.squeeze(),
    }

# Preprocess dataset
tokenized_dataset = train_dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=train_dataset.column_names,
)


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

## 7. Setup Training Arguments


In [8]:
training_args = TrainingArguments(
    output_dir="./bank-model-2b-finetuned",
    num_train_epochs=3,
    per_device_train_batch_size=1,  # Giảm nếu hết memory
    gradient_accumulation_steps=8,  # Tăng để mô phỏng batch size lớn hơn
    learning_rate=2e-4,
    fp16=False,  # Sử dụng bfloat16 thay vì fp16
    bf16=True,
    logging_steps=10,
    save_steps=100,
    eval_steps=100,
    save_total_limit=3,
    remove_unused_columns=False,
    push_to_hub=False,  # Set True nếu muốn push lên Hugging Face
    hub_model_id="hainguyen306201/bank-model-2b",  # Model ID trên Hugging Face
    report_to="tensorboard",
    warmup_steps=100,
    max_grad_norm=0.3,
    lr_scheduler_type="cosine",
    optim="paged_adamw_8bit",  # Optimizer tiết kiệm memory
)


## 8. Tạo Data Collator


In [9]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=processor.tokenizer,
    mlm=False,  # Không dùng masked language modeling
)


## 9. Khởi tạo Trainer


In [10]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
)


## 10. Bắt đầu Training


In [11]:
# Training
print("Bắt đầu training...")
trainer.train()
print("Training hoàn thành!")


Bắt đầu training...


IndexError: too many indices for tensor of dimension 1

## 11. Lưu Model


In [ ]:
# Lưu model đã fine-tune
trainer.save_model("./bank-model-2b-finetuned")
processor.save_pretrained("./bank-model-2b-finetuned")

print("Model đã được lưu!")


## 12. Test Model sau Training


In [ ]:
# Load model đã fine-tune để test
test_model = Qwen3VLForConditionalGeneration.from_pretrained(
    "./bank-model-2b-finetuned",
    device_map="auto",
    torch_dtype=torch.bfloat16,
)

test_processor = AutoProcessor.from_pretrained("./bank-model-2b-finetuned")

# Test với một ảnh mẫu
messages = [
    {
        "role": "user",
        "content": [
            {"type": "image", "image": "https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen-VL/assets/demo.jpeg"},
            {"type": "text", "text": "Mô tả ảnh này."},
        ],
    }
]

inputs = test_processor.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_dict=True,
    return_tensors="pt"
)
inputs = inputs.to(test_model.device)

generated_ids = test_model.generate(**inputs, max_new_tokens=128)
generated_ids_trimmed = [
    out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = test_processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)

print("Output:", output_text)


## 13. Upload lên Hugging Face (Optional)


In [ ]:
# Uncomment để upload model lên Hugging Face
# trainer.push_to_hub()
# hoặc
# from huggingface_hub import HfApi
# api = HfApi()
# api.upload_folder(
#     folder_path="./bank-model-2b-finetuned",
#     repo_id="hainguyen306201/bank-model-2b",
#     repo_type="model",
# )
